<a href="https://colab.research.google.com/github/cristinaluna/satellite_passes/blob/main/satellite_passes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.5/234.5 kB 13.0 MB/s eta 0:00:00


In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests
from skyfield.api import wgs84, load
import math
import datetime
import pytz
import csv
from collections import Counter


OBS_LAT = 40.481
OBS_LON = -3.364
TLE_URL = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
TLE_LOCAL_FILE = 'satellites.tle'

MIN_INCL_DEG = 70.0
MIN_ELEV_DEG = 70.0
MAX_ELEV_DEG = 90.0

LOCAL_TZ = pytz.timezone('Europe/Madrid')

SAT_INFO = {

    "GPS (Constelación NAVSTAR)": {
        "descripcion": (
            "Constelación estadounidense de navegación global (aprox. 31 satélites en MEO). "
            "Provee servicios de posicionamiento y tiempo a nivel mundial."
        ),
        "frecuencia": (
            "L1 (1575.42 MHz: C/A, P(Y)), "
            "L2 (1227.60 MHz: P(Y)), "
            "L5 (1176.45 MHz: señal civil)."
        ),
        "modo": "CDMA (señales C/A, P(Y), M, etc.)"
    },
    "GLONASS": {
        "descripcion": (
            "Sistema ruso de navegación global (24 satélites activos en ~19.100 km de altitud)."
        ),
        "frecuencia": (
            "L1 ~1602 MHz (FDMA), "
            "L2 ~1246 MHz (FDMA), "
            "L3 ~1202 MHz (CDMA, nuevo estándar)."
        ),
        "modo": "FDMA tradicional + transiciones a CDMA (GLONASS-K)."
    },
    "Galileo": {
        "descripcion": (
            "Constelación europea (ESA) de navegación global (24 satélites + reserva). "
            "Órbitas MEO a ~23.222 km."
        ),
        "frecuencia": (
            "E1 (1575.42 MHz), "
            "E5a (1176.45 MHz), "
            "E5b (1207.14 MHz), "
            "E6 (1278.75 MHz)."
        ),
        "modo": "CDMA (señales Open Service, PRS, Commercial Service, etc.)"
    },
    "BeiDou/Compass": {
        "descripcion": (
            "Constelación china (BDS). Combina satélites MEO, IGSO y GEO. "
            "Provee cobertura regional y global."
        ),
        "frecuencia": (
            "B1I (1561.098 MHz), B1C (1575.42 MHz), "
            "B2I (1207.14 MHz), B2a (1176.45 MHz), etc."
        ),
        "modo": "CDMA (distintas señales civiles y militares)."
    },
    "NavIC (IRNSS)": {
        "descripcion": (
            "Sistema de navegación regional de la India (ISRO). Consta de 7 satélites (GEO/IGSO)."
        ),
        "frecuencia": (
            "L5 (1176.45 MHz), "
            "S-band (~2492.028 MHz)."
        ),
        "modo": "CDMA"
    },
    "QZSS": {
        "descripcion": (
            "Quasi-Zenith Satellite System, Japón (JAXA). Cubierta regional para Asia-Pacífico. "
            "Satélites en órbitas QZO + GEO."
        ),
        "frecuencia": (
            "L1 C/A (1575.42 MHz), L2 (1227.60 MHz), "
            "L5 (1176.45 MHz), L6 (1278.75 MHz)."
        ),
        "modo": "CDMA (soporta interoperabilidad con GPS)."
    },


    "Landsat 8": {
        "descripcion": (
            "Satélite de observación terrestre (NASA/USGS) lanzado en 2013. "
            "Órbita heliosíncrona ~705 km."
        ),
        "frecuencia": "X-band (downlink ~8.0 GHz para telemetría e imágenes)",
        "modo": "Telemetría / Downlink de imágenes (GeoTIFF)."
    },
    "Landsat 9": {
        "descripcion": (
            "Última generación de la serie Landsat (lanzada en 2021). "
            "Misiones de monitoreo global."
        ),
        "frecuencia": "X-band (downlink ~8.0 GHz)",
        "modo": "Telemetría / Downlink de imágenes"
    },
    "Sentinel-1": {
        "descripcion": (
            "Radar de apertura sintética (SAR) de la ESA. "
            "Constelación con Sentinel-1A y 1B (1B inoperativo desde 2022)."
        ),
        "frecuencia": "C-band (~5.405 GHz)",
        "modo": "SAR (banda C)."
    },
    "Sentinel-2": {
        "descripcion": (
            "Satélites ópticos multiespectrales (ESA). Órbita heliosíncrona ~786 km. "
            "Resolución ~10 m (banda visible)."
        ),
        "frecuencia": "X-band (~8 GHz downlink).",
        "modo": "Telemetría (imágenes multiespectrales)."
    },
    "Sentinel-3": {
        "descripcion": (
            "Satélites para oceanografía, temperatura superficial y altimetría (ESA/EUMETSAT)."
        ),
        "frecuencia": "X-band (downlink), S-band (TT&C).",
        "modo": "Telemetría (radiómetro, altímetro, etc.)"
    },
    "Sentinel-4": {
        "descripcion": (
            "Instrumento a bordo de un satélite Meteosat de órbita geoestacionaria. "
            "Monitoreo atmosférico (ESA)."
        ),
        "frecuencia": "Canal de carga en Meteosat (datos no abiertos).",
        "modo": "Espectrómetro (GEO)."
    },
    "Sentinel-5": {
        "descripcion": (
            "Instrumento a bordo de satélites MetOp (órbita polar). "
            "Monitoreo de gases atmosféricos (ESA)."
        ),
        "frecuencia": "Instalado en MetOp (varias bandas), telemetría no pública.",
        "modo": "Espectrómetro"
    },
    "Sentinel-6": {
        "descripcion": (
            "Altímetro de precisión para medición de nivel del mar (misión conjunta ESA/NASA)."
        ),
        "frecuencia": "X-band (downlink ~8 GHz), S-band (TT&C).",
        "modo": "Telemetría (altimetría)."
    },
    "GOES-16": {
        "descripcion": (
            "Satélite meteorológico geoestacionario de EE.UU. (NOAA). "
            "Ubicado en ~75.2°O."
        ),
        "frecuencia": (
            "L-band (HRIT/EMWIN ~1.69 GHz), "
            "S-band (GVAR, ~2.0-2.3 GHz)."
        ),
        "modo": "Imágenes y datos meteorológicos (Geo)."
    },
    "GOES-17": {
        "descripcion": (
            "Satélite meteorológico geoestacionario de EE.UU. (NOAA). "
            "Ubicado en ~137.2°O."
        ),
        "frecuencia": "L-band y S-band similares a GOES-16 (GVAR/HRIT).",
        "modo": "No aplica (geo meteorología)."
    },
    "Meteosat": {
        "descripcion": (
            "Serie europea de satélites meteorológicos (EUMETSAT). "
            "GEO a 0° y 9.5°E (Meteosat-11, etc.)."
        ),
        "frecuencia": "L-band (HRIT/LRIT) ~1.69 GHz.",
        "modo": "Transmisión meteorológica GEO."
    },
    "Himawari-8": {
        "descripcion": "Satélite meteorológico de Japón (JMA), GEO en 140.7°E.",
        "frecuencia": "HRIT/LRIT (L-band ~1.69 GHz)",
        "modo": "Imágenes meteorológicas"
    },
    "Himawari-9": {
        "descripcion": "Satélite gemelo de Himawari-8, operado por JMA (GEO).",
        "frecuencia": "HRIT/LRIT en banda L",
        "modo": "Imágenes meteorológicas"
    },
    "Feng Yun (serie)": {
        "descripcion": (
            "Satélites meteorológicos chinos (CMA). Incluye FY-2 (GEO), FY-3 (LEO) y FY-4 (GEO)."
        ),
        "frecuencia": (
            "FY-3 (LEO): ~137 MHz (APT, HRPT), ~1700 MHz (HRPT), X-band. "
            "FY-4 (GEO): L-band HRIT."
        ),
        "modo": "Meteorología"
    },
    "CARTOSAT": {
        "descripcion": (
            "Serie de satélites de observación de la India (ISRO). "
            "Órbita polar/heliosíncrona ~630-720 km."
        ),
        "frecuencia": "X-band (~8 GHz downlink de imágenes)",
        "modo": "Telemetría"
    },

    "Intelsat (serie)": {
        "descripcion": (
            "Flota internacional en GEO (Intelsat). Cobertura global de TV, datos y telefonía."
        ),
        "frecuencia": "C (~4/6 GHz), Ku (~11/14 GHz), Ka (~20/30 GHz) band.",
        "modo": "Telecom (TV, VSAT, datos)."
    },
    "Eutelsat (serie)": {
        "descripcion": (
            "Satélites de comunicaciones europeos (Eutelsat). "
            "Órbitas GEO en posiciones varias (13°E, 9°E...)."
        ),
        "frecuencia": "Ku (~12 GHz), Ka (~20/30 GHz) band.",
        "modo": "TV satelital, datos, banda ancha."
    },
    "SES (serie)": {
        "descripcion": (
            "Flota de SES (Luxemburgo). GEO (Astra), O3b MEO, etc. "
            "Servicios de TV y banda ancha."
        ),
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "Iridium NEXT": {
        "descripcion": (
            "Constelación LEO (~780 km) para telefonía y datos satelitales. "
            "66 satélites + repuestos."
        ),
        "frecuencia": (
            "L-band (~1.6 GHz) para usuario, "
            "Ka-band (19/29 GHz) para inter-satélite."
        ),
        "modo": "Telefonía/datos (TDMA/CDMA)."
    },
    "Starlink": {
        "descripcion": (
            "Constelación LEO de SpaceX. Miles de satélites para Internet "
            "de banda ancha global."
        ),
        "frecuencia": "Ku (~12 GHz) y Ka (~20-30 GHz), en desarrollo láser inter-satélite.",
        "modo": "Datos (banda ancha)."
    },
    "OneWeb": {
        "descripcion": (
            "Constelación LEO de banda ancha (Reino Unido/India). "
            "Satélites a ~1200 km."
        ),
        "frecuencia": "Ku-band (~12/14 GHz).",
        "modo": "Datos (banda ancha)."
    },

    "Hubble Space Telescope": {
        "descripcion": (
            "Telescopio espacial NASA/ESA, en órbita ~540 km. "
            "Observación en visible, UV, IR cercano."
        ),
        "frecuencia": "S-band (comandos ~2 GHz), Ku-band (~14 GHz datos).",
        "modo": "Telemetría científica"
    },
    "James Webb Space Telescope": {
        "descripcion": (
            "Telescopio espacial en L2 (1.5 millones km de la Tierra), "
            "colaboración NASA/ESA/CSA."
        ),
        "frecuencia": "Ka-band (~25.9 GHz downlink), S-band (~2 GHz uplink).",
        "modo": "Telemetría/Comunicaciones de ciencia"
    },
    "GAIA": {
        "descripcion": (
            "Observatorio astrométrico de la ESA, en L2. "
            "Cartografía de precisión de la Vía Láctea."
        ),
        "frecuencia": "X-band (~8 GHz) / Ka-band para datos de alta tasa.",
        "modo": "Telemetría científica"
    },
    "XMM-Newton": {
        "descripcion": (
            "Observatorio europeo de rayos X. Órbita alta elíptica (~48 h)."
        ),
        "frecuencia": "X-band (~8 GHz).",
        "modo": "Telemetría"
    },
    "Chandra X-ray Observatory": {
        "descripcion": (
            "Observatorio de rayos X de la NASA. Órbita altamente elíptica (~64 h)."
        ),
        "frecuencia": "S-band (comandos ~2 GHz), K-band (datos ~26 GHz).",
        "modo": "Telemetría"
    },
    "TESS": {
        "descripcion": (
            "Satélite cazador de exoplanetas (NASA). Órbita P/2 resonante con la Luna."
        ),
        "frecuencia": "Ka-band (~26 GHz) para datos científicos.",
        "modo": "Telemetría"
    },


    "Estación Espacial Internacional (ISS)": {
        "descripcion": (
            "Estación Espacial tripulada (EE.UU., Rusia, Europa, Japón, Canadá). "
            "Órbita ~400 km."
        ),
        "frecuencia": (
            "145.800 MHz (downlink FM voz), "
            "437.550 MHz (APRS), "
            "Varios enlaces Ku/S-band para payloads."
        ),
        "modo": "FM/APRS (uso amateur), también banda Ku/S para datos."
    },
    "Tiangong": {
        "descripcion": (
            "Estación Espacial China (CSS). Módulos Tianhe, Wentian, Mengtian. "
            "Órbita LEO ~390-400 km."
        ),
        "frecuencia": (
            "145.825 MHz (posibles pruebas amateur), "
            "Enlaces de control en S-band, datos en Ku."
        ),
        "modo": "FM (experimental), telemetría profesional"
    },

    "KH-11 KENNEN (serie)": {
        "descripcion": (
            "Satélites espía ópticos de EE.UU. (NRO). Órbita polar ~250-1000 km (clasificado)."
        ),
        "frecuencia": "Clasificado (posiblemente en banda X/S).",
        "modo": "Reconocimiento óptico"
    },
    "Helios (serie)": {
        "descripcion": (
            "Satélites de observación militar de Francia. Helios 2 (~700 km)."
        ),
        "frecuencia": "Clasificado (X-band downlink).",
        "modo": "Reconocimiento"
    },
    "SAR-Lupe": {
        "descripcion": (
            "Constelación alemana de reconocimiento radar (SAR). "
            "Órbita polar ~500 km."
        ),
        "frecuencia": "X-band (SAR). Posible downlink en banda X clasificado.",
        "modo": "SAR militar"
    },
    "COSMO-SkyMed": {
        "descripcion": (
            "Constelación italiana de observación radar (~620 km). "
            "Uso civil y militar."
        ),
        "frecuencia": "X-band (SAR), downlink codificado.",
        "modo": "Reconocimiento"
    },
    "Yaogan (serie)": {
        "descripcion": (
            "Satélites chinos de reconocimiento (SAR, óptico, ELINT). "
            "Varias órbitas LEO."
        ),
        "frecuencia": "Clasificado (posibles bandas X, S, etc.)",
        "modo": "Reconocimiento"
    },
    "Ofek (serie)": {
        "descripcion": (
            "Satélites espía de Israel (IAI). Órbita retrograda ~400-600 km."
        ),
        "frecuencia": "Clasificado (bandas S/X).",
        "modo": "Reconocimiento"
    },

    "OCO-2": {
        "descripcion": (
            "Orbitador de Carbono (NASA). Mide CO2 en la atmósfera. "
            "Órbita heliosíncrona ~705 km."
        ),
        "frecuencia": "X-band (~8 GHz) para envío de datos.",
        "modo": "Telemetría"
    },
    "CALIPSO": {
        "descripcion": (
            "Satélite conjunto NASA/CNES para estudio de aerosoles y nubes. "
            "Constelación A-Train."
        ),
        "frecuencia": "X-band (downlink), láser de 532/1064 nm para LIDAR.",
        "modo": "Telemetría"
    },
    "DMSP (serie)": {
        "descripcion": (
            "Programa militar meteorológico de EE.UU. (Defense Meteorological). "
            "Órbita polar ~830 km."
        ),
        "frecuencia": "Algunas bandas clasificadas; S-band (TT&C).",
        "modo": "Telemetría (militar)."
    },
    "CloudSat": {
        "descripcion": (
            "Radar de nubes (NASA). Parte de la A-Train (~705 km). "
            "94 GHz Cloud Profiling Radar."
        ),
        "frecuencia": (
            "X-band (downlink ~8 GHz), "
            "Radar a 94 GHz para estudio de nubes."
        ),
        "modo": "Telemetría"
    },

    "AsiaSat (serie)": {
        "descripcion": (
            "Satélites de AsiaSat (Hong Kong). Posiciones GEO (100.5°E, etc.)."
        ),
        "frecuencia": "C/Ku-band",
        "modo": "Telecom (TV, datos)."
    },
    "MEASAT (serie)": {
        "descripcion": (
            "Satélites de Malasia (Measat Systems). Orbita GEO ~91.5°E, etc."
        ),
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Arabsat (serie)": {
        "descripcion": (
            "Flota de satélites de la Liga Árabe (Arabsat/Badr) en GEO (26°E, 30.5°E...)."
        ),
        "frecuencia": "C/Ku/Ka-band",
        "modo": "Telecom"
    },
    "Türksat (serie)": {
        "descripcion": (
            "Satélites GEO de telecomunicaciones turcos (42°E, 50°E...)."
        ),
        "frecuencia": "Ku-band",
        "modo": "Telecom"
    },
    "Sky Perfect JSAT": {
        "descripcion": (
            "Operador satelital japonés (JSAT). Satélites en GEO para Asia-Pacífico."
        ),
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Telstar (serie)": {
        "descripcion": (
            "Satélites de Telesat (Canadá). Orbita GEO (14°W, etc.) y algunos en LEO futuros."
        ),
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "DirecTV (serie)": {
        "descripcion": "Satélites GEO de televisión directa (EE.UU.) ~99°W, 101°W, etc.",
        "frecuencia": "Ku (~12 GHz), Ka (~20/30 GHz) band.",
        "modo": "Telecom (TV DTH)"
    },
    "Amazonas (serie)": {
        "descripcion": (
            "Satélites de Hispasat (España/Latinoamérica), positions GEO: ~61°W, 74°W."
        ),
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "O3b (constelación)": {
        "descripcion": (
            "Constelación MEO de SES (~8000 km), banda ancha en órbita ecuatorial."
        ),
        "frecuencia": "Ka-band (~20/30 GHz)",
        "modo": "Datos"
    },
    "Yamal (serie)": {
        "descripcion": "Satélites GEO de Rusia (Gazprom), posiciones ~49°E, 90°E, 103.5°E.",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Badr (serie)": {
        "descripcion": (
            "También llamados Arabsat en 26°E (Badr-4, -5, -6, -7...). Cobertura MENA/Europa."
        ),
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Chinasat (serie)": {
        "descripcion": (
            "Satélites de comunicación chinos (Zhongxing), en posiciones GEO diversas."
        ),
        "frecuencia": "C/Ku/Ka-band",
        "modo": "Telecom"
    },
    "Koreasat (serie)": {
        "descripcion": (
            "Satélites GEO de telecom. surcoreanos (posiciones 113°E, 116°E...)."
        ),
        "frecuencia": "Ku-band",
        "modo": "Telecom"
    },
    "VSAT (serie)": {
        "descripcion": (
            "Término genérico para terminales de apertura muy pequeña. "
            "No un satélite en sí, sino soluciones en C/Ku/Ka."
        ),
        "frecuencia": "Ku/Ka/C-band (varía según proveedor).",
        "modo": "Telecom (pequeñas antenas)."
    },

    "RADARSAT Constellation": {
        "descripcion": (
            "Constelación canadiense de 3 satélites SAR (lanzados en 2019) ~600 km, "
            "banda C."
        ),
        "frecuencia": "C-band (SAR ~5.405 GHz)",
        "modo": "SAR"
    },
    "TerraSAR-X": {
        "descripcion": (
            "Satélite alemán de observación radar (X-band ~9.65 GHz), ~514 km."
        ),
        "frecuencia": "X-band (SAR ~9.65 GHz)",
        "modo": "SAR"
    },
    "SPOT (serie)": {
        "descripcion": (
            "Satélites de observación franceses (Airbus). "
            "SPOT-6/7 en ~694 km heliosíncrona."
        ),
        "frecuencia": "X-band (~8 GHz downlink)",
        "modo": "Telemetría de imágenes"
    },
    "ResourceSat": {
        "descripcion": (
            "Satélites de observación indios (ISRO). ResourceSat-2 en ~820 km. "
            "Cámaras LISS, AWiFS."
        ),
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "KOMPSAT (serie)": {
        "descripcion": (
            "Satélites de observación surcoreanos (Arirang). "
            "Algunos con óptico, otros con SAR."
        ),
        "frecuencia": "X-band (downlink), S-band (TT&C).",
        "modo": "Telemetría"
    },
    "Gaofen (serie)": {
        "descripcion": (
            "Satélites chinos de alta resolución (óptico, SAR). "
            "Órbitas heliosíncronas ~600-700 km."
        ),
        "frecuencia": "X-band downlink (~8 GHz).",
        "modo": "Telemetría"
    },
    "ALOS (serie)": {
        "descripcion": (
            "Advanced Land Observing Satellite (JAXA). ALOS-2 usa L-band SAR (~1.2 GHz)."
        ),
        "frecuencia": (
            "L-band SAR (~1.2 GHz), X-band downlink para datos."
        ),
        "modo": "SAR/Óptico"
    },
    "TanDEM-X": {
        "descripcion": (
            "Misión alemana (DLR) junto con TerraSAR-X para interferometría X-band."
        ),
        "frecuencia": "X-band (SAR ~9.65 GHz)",
        "modo": "SAR"
    },
    "WorldView (serie)": {
        "descripcion": (
            "Satélites de DigitalGlobe/Maxar (WV-1,2,3,4). "
            "Resolución óptica alta (~0.31 m)."
        ),
        "frecuencia": "X-band (downlink ~8 GHz)",
        "modo": "Telemetría imágenes"
    },
    "Planet Labs (constelación)": {
        "descripcion": (
            "Cubesats Dove, SuperDove en LEO ~475-500 km. "
            "Imágenes diarias de la Tierra."
        ),
        "frecuencia": "UHF (telemetría), X-band (downlink imágenes).",
        "modo": "Telemetría"
    },
    "RapidEye (constelación)": {
        "descripcion": (
            "Constelación alemana (5 satélites). Ahora operada por Planet. "
            "Banda multiespectral ~6.5 m/píxel."
        ),
        "frecuencia": "S-band (TT&C), X-band (data downlink).",
        "modo": "Telemetría"
    },
    "SkySat (constelación)": {
        "descripcion": (
            "Satélites de alta resolución (Planet). Resolución submétrica. "
            "Órbita ~500 km."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Telemetría"
    },
    "NigeriaSat": {
        "descripcion": (
            "Satélites de observación de Nigeria (NASRDA). Órbita LEO ~700 km."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Telemetría"
    },
    "VNREDSat": {
        "descripcion": (
            "Satélite de observación de Vietnam (~680 km heliosíncrona). "
            "Resolución ~2.5 m."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Telemetría"
    },

    "Solar and Heliospheric Observatory (SOHO)": {
        "descripcion": (
            "Observatorio solar ESA/NASA, en halo orbit L1 (~1.5 millones km)."
        ),
        "frecuencia": "X-band (DSN), ~8.4 GHz para datos.",
        "modo": "Telemetría ciencia"
    },
    "Spektr-RG": {
        "descripcion": (
            "Observatorio ruso-alemán de rayos X (ART-XC, eROSITA), en L2."
        ),
        "frecuencia": "X-band (~8 GHz).",
        "modo": "Telemetría"
    },
    "Chandrayaan-2": {
        "descripcion": (
            "Misión lunar india (ISRO). Orbitador a ~100 km sobre la Luna."
        ),
        "frecuencia": "S-band (~2.2-2.3 GHz) y X-band (~8 GHz).",
        "modo": "Telemetría"
    },
    "BepiColombo": {
        "descripcion": (
            "Sonda ESA/JAXA rumbo a Mercurio. Varias asistencias gravitatorias."
        ),
        "frecuencia": (
            "X-band (~8.4 GHz) para telecom, Ka-band (~32 GHz) para "
            "experimentos de radio-ciencia."
        ),
        "modo": "Telemetría ciencia"
    },
    "Solar Orbiter": {
        "descripcion": (
            "Sonda ESA/NASA para estudio del Sol. Órbita elíptica con perihelio <0.3 UA."
        ),
        "frecuencia": (
            "X-band (~8.4 GHz) principal, algo en Ka-band (~32 GHz)."
        ),
        "modo": "Telemetría"
    },
    "DAMPE": {
        "descripcion": (
            "Dark Matter Particle Explorer (China), en LEO ~500 km. "
            "Detecta rayos cósmicos y gamma."
        ),
        "frecuencia": "S-band (TT&C), X-band (downlink datos).",
        "modo": "Telemetría"
    },
    "Astrosat": {
        "descripcion": (
            "Observatorio de la India (ISRO), multi-longitud de onda (UV, X). "
            "Órbita ~650 km."
        ),
        "frecuencia": "S-band (comandos), X-band (~8 GHz) datos.",
        "modo": "Telemetría"
    },
    "ADITYA-L1": {
        "descripcion": (
            "Misión solar india (ISRO) a punto L1, estudio de la corona solar."
        ),
        "frecuencia": "S/X-band (~2.2 GHz / ~8 GHz).",
        "modo": "Telemetría"
    },
    "ExoMars Trace Gas Orbiter": {
        "descripcion": (
            "Orbitador marciano ESA/Roscosmos. Lanza y recibe en X-band, "
            "relay con rover en UHF."
        ),
        "frecuencia": "X-band (~8.4 GHz) + UHF relay (~400 MHz).",
        "modo": "Telemetría"
    },

    "MetOp (serie)": {
        "descripcion": (
            "Satélites polares de EUMETSAT (MetOp-A/B/C) ~817 km. "
            "Instrumentos IASI, AMSU, etc."
        ),
        "frecuencia": "L-band (~1.5 GHz), X-band (~8 GHz downlink).",
        "modo": "Telemetría meteorológica"
    },
    "NOAA (serie)": {
        "descripcion": (
            "Satélites meteorológicos polares de EE.UU. (ej. NOAA-18,19,20...). "
            "APT ~137 MHz en algunos antiguos."
        ),
        "frecuencia": "137 MHz (APT en NOAA-15/18/19), L-band HRPT, etc.",
        "modo": "APT/HRPT (antiguos), JPSS en banda superior"
    },
    "GEO-KOMPSAT": {
        "descripcion": (
            "Satélites meteorológicos geoestacionarios de Corea del Sur (GEO-KOMPSAT-2A/B)."
        ),
        "frecuencia": "L-band (HRIT), S-band (LRIT), etc.",
        "modo": "Meteorología"
    },
    "INSAT (serie)": {
        "descripcion": (
            "Satélites de India (ISRO) en GEO. Uso mixto: telecom, meteorología."
        ),
        "frecuencia": "C/Extended-C/Ku-band",
        "modo": "Telecom/meteorología"
    },
    "Electro-L (serie)": {
        "descripcion": (
            "Satélites meteorológicos rusos en GEO (Ej. Electro-L 2 a 77.8°E)."
        ),
        "frecuencia": "HRIT/LRIT en L-band, etc.",
        "modo": "No aplica (imágenes meteorológicas)"
    },
    "FY-4 (serie)": {
        "descripcion": (
            "Satélites meteorológicos chinos en GEO (~105°E). "
            "Imágenes en banda visible, IR."
        ),
        "frecuencia": "L-band (downlink HRIT), etc.",
        "modo": "No aplica"
    },
    "COSMIC-2 (constelación)": {
        "descripcion": (
            "Constelación Taiwán/EE.UU. para radio-ocultación GNSS. "
            "Órbita ecuatorial ~520 km."
        ),
        "frecuencia": (
            "Recepción de GNSS (L1/L2), S-band downlink (~2 GHz)."
        ),
        "modo": "Datos (RO)"
    },

    "AMSAT (serie)": {
        "descripcion": (
            "Satélites de radioaficionados de diversas organizaciones (AO-7, AO-73, FO-29...)."
        ),
        "frecuencia": (
            "VHF/UHF/SHF, por ejemplo 145 MHz uplink, 435 MHz downlink, etc. "
            "Depende del satélite."
        ),
        "modo": "FM, SSB, CW, Digital (varía)"
    },
    "FUNcube (serie)": {
        "descripcion": (
            "Satélites educativos de AMSAT-UK (FUNcube-1, FUNcube-2...). "
            "Órbita LEO ~600-700 km."
        ),
        "frecuencia": (
            "435.150 MHz (downlink SSB), 145.915 MHz (uplink), "
            "o similar, según versión."
        ),
        "modo": "SSB/FM, Telemetría BPSK"
    },
    "BY70 (serie)": {
        "descripcion": (
            "Pequeños satélites chinos de radioaficionados. "
            "Órbita LEO ~500 km."
        ),
        "frecuencia": "UHF/VHF (varía), por ejemplo 436.xxx MHz.",
        "modo": "FM"
    },
    "LilacSat (serie)": {
        "descripcion": (
            "Proyectos de la Universidad de Harbin, China. "
            "Órbita LEO ~500 km."
        ),
        "frecuencia": (
            "VHF/UHF: 145.xxx / 435.xxx, "
            "varios experimentos digitales."
        ),
        "modo": "FM/Transpondedor digital"
    },
    "ES'HAIL-2": {
        "descripcion": (
            "También llamado QO-100 (Qatar-OSCAR 100). "
            "Primer satélite geoaficionado en 25.5°E (GEO)."
        ),
        "frecuencia": "Uplink 2400 MHz, Downlink 10489 MHz.",
        "modo": "SSB/CW, DATV (banda estrecha y ancha)"
    },

    "COSPAS-SARSAT": {
        "descripcion": (
            "Sistema internacional de búsqueda y rescate. "
            "Balizas 406 MHz detectadas por satélites LEO/GEO/MEO."
        ),
        "frecuencia": "406 MHz (balizas), 121.5 MHz (aux).",
        "modo": "SAR (coordinado a estaciones de tierra)"
    },
    "GOES-R (componente SAR)": {
        "descripcion": (
            "GOES con transpondedor MEOSAR para 406 MHz. "
            "También monitoreo de balizas."
        ),
        "frecuencia": "406 MHz (balizas).",
        "modo": "SAR"
    },
    "Galileo (componente SAR)": {
        "descripcion": (
            "Servicio SAR integrado en Galileo (MEOSAR). "
            "Detección balizas 406 MHz y retransmisión L-band."
        ),
        "frecuencia": "406 MHz (balizas), L-band para retorno.",
        "modo": "SAR"
    },
    "GLONASS (componente SAR)": {
        "descripcion": (
            "Segmento SAR en satélites GLONASS (CDM-MEOSAR). "
            "Compatible con 406 MHz."
        ),
        "frecuencia": "406 MHz (balizas).",
        "modo": "SAR"
    },

    "Project Kuiper": {
        "descripcion": (
            "Constelación de internet de Amazon (en desarrollo), "
            "~3236 satélites en LEO."
        ),
        "frecuencia": "Ka-band (~26-40 GHz).",
        "modo": "Datos (banda ancha)"
    },
    "AST SpaceMobile (BlueWalker 3)": {
        "descripcion": (
            "Satélite demo para telefonía directa al móvil (4G/5G). "
            "Órbita LEO ~500 km."
        ),
        "frecuencia": "Bandas celulares (700-2100 MHz), satelitales en S/Ka.",
        "modo": "Datos/Telefonía"
    },
    "Capella Space": {
        "descripcion": (
            "Constelación comercial SAR (EE.UU.). Orbita LEO ~500-600 km. "
            "Resolución submétrica en X-band."
        ),
        "frecuencia": "X-band (SAR ~9.65 GHz).",
        "modo": "SAR"
    },
    "ICEYE": {
        "descripcion": (
            "Constelación comercial SAR (Finlandia). "
            "Pequeños satélites LEO con radar X-band."
        ),
        "frecuencia": "X-band (SAR ~9.65 GHz).",
        "modo": "SAR"
    },
    "GHGSat": {
        "descripcion": (
            "Monitoreo de gases de efecto invernadero (Canadá). "
            "LEO ~500 km."
        ),
        "frecuencia": "X-band (downlink ~8 GHz).",
        "modo": "Telemetría"
    },
    "Satellogic (NewSat)": {
        "descripcion": (
            "Constelación argentina de observación (~475-500 km). "
            "Cámaras de alta resolución."
        ),
        "frecuencia": "UHF (telemetría), X-band (downlink imágenes).",
        "modo": "Telemetría"
    },
    "Astroscale (ELSA-d)": {
        "descripcion": (
            "Demostrador de eliminación de desechos espaciales. "
            "Tecnología de acoplamiento y deorbit."
        ),
        "frecuencia": "S-band (~2 GHz) / X-band (~8 GHz).",
        "modo": "Demostración, telemetría"
    },

    "KINEIS-5E": {
        "descripcion": (
            "Satélite de la constelación Kineis (Francia) para IoT y conectividad global, "
            "en órbita LEO ~650 km."
        ),
        "frecuencia": "UHF/VHF (mensajería IoT), S-band (posible TT&C).",
        "modo": "IoT/Telemetría"
    },

    "HEAD-1": {
        "descripcion": (
            "Posible satélite de HEAD Aerospace (China) para servicios satelitales. "
            "Se desconoce órbita exacta."
        ),
        "frecuencia": "Datos limitados (posible banda UHF o S-band).",
        "modo": "Datos (monitoreo/telecom)."
    },

    "DUBAISAT-2": {
        "descripcion": (
            "Satélite de observación terrestre (Emiratos Árabes). Órbita ~600 km "
            "heliosíncrona, lanzado en 2013."
        ),
        "frecuencia": "X-band (downlink de imágenes), S-band TT&C.",
        "modo": "Telemetría / Imágenes"
    },

    "IRIDIUM 100": {
        "descripcion": (
            "Parte de la constelación Iridium para telefonía/datos. "
            "Satélite LEO ~780 km."
        ),
        "frecuencia": "L-band (~1.6 GHz para usuario), Ka-band para inter-satélite.",
        "modo": "Telefonía/datos"
    },

    "AEROCUBE 6B": {
        "descripcion": (
            "CubeSat experimental del Laboratorio Aeroespacial (The Aerospace Corp). "
            "Órbita LEO ~600-650 km."
        ),
        "frecuencia": "Posible UHF/VHF para telemetría y control.",
        "modo": "Demostración / Telemetría"
    },

    "IRIDIUM 133": {
        "descripcion": (
            "Otro satélite de la constelación Iridium (2ª generación o 'NEXT' si es reciente)."
        ),
        "frecuencia": "L-band, Ka-band (inter-satélite).",
        "modo": "Telefonía/datos"
    },

    "SHIYAN 6 03 (SY-6 03)": {
        "descripcion": (
            "Satélite chino 'Shiyan' (experimentales). A menudo asociados a verificación "
            "tecnológica en LEO."
        ),
        "frecuencia": "Clasificada o limitada, posiblemente S/X-band para telemetría.",
        "modo": "Pruebas tecnológicas / Reconocimiento"
    },

    "YUNHAI-3 01": {
        "descripcion": (
            "Satélite meteorológico/científico chino de la serie Yunhai, "
            "para estudios atmosféricos en LEO."
        ),
        "frecuencia": "L-band, S-band o X-band (datos meteorológicos).",
        "modo": "Meteorología / Monitoreo atmosférico"
    },

    "TIANHUI 2-01A": {
        "descripcion": (
            "Parte de la serie Tianhui de mapeo/cartografía (China). "
            "Órbita LEO heliosíncrona ~500 km."
        ),
        "frecuencia": "X-band (downlink de imágenes), S-band TT&C.",
        "modo": "Cartografía / Observación"
    },

    "TIANHUI 2-01B": {
        "descripcion": (
            "Satélite compañero de 2-01A para misiones en formación (China). "
            "Observación estereoscópica."
        ),
        "frecuencia": "X-band (downlink), S-band TT&C.",
        "modo": "Cartografía"
    },

    "TIANHUI 5D": {
        "descripcion": (
            "Posible satélite de la serie Tianhui (mapeo topográfico). "
            "Datos detallados no públicos."
        ),
        "frecuencia": "X-band (para imágenes).",
        "modo": "Telemetría / Reconocimiento"
    },

    "TIANHUI 5C": {
        "descripcion": "Otro satélite chino de la serie Tianhui para cartografía.",
        "frecuencia": "X-band (imágenes), S-band (control).",
        "modo": "Observación"
    },

    "YUNHAI-1 01": {
        "descripcion": (
            "Satélite chino para estudios meteorológicos y ionosféricos, "
            "serie Yunhai-1."
        ),
        "frecuencia": "S-band o X-band para downlink meteorológico.",
        "modo": "Meteorología / Ciencia"
    },

    "WNISAT 1R": {
        "descripcion": (
            "Satélite japonés de la Universidad de Wakayama/NASA. "
            "Monitorea hielos polares, clima."
        ),
        "frecuencia": "UHF/VHF para telemetría, posible S-band data.",
        "modo": "Telemetría"
    },

    "SHIJIAN-6 03B (SJ-6 03B)": {
        "descripcion": (
            "Parte de la familia Shijian (misiones experimentales chinas). "
            "Órbita LEO."
        ),
        "frecuencia": "Clasificado, usualmente S-band / X-band.",
        "modo": "Investigación / Reconocimiento"
    },

    "RCM-3": {
        "descripcion": (
            "Radarsat Constellation Mission - 3 (Canadá). "
            "SAR en banda C (~5.4 GHz)."
        ),
        "frecuencia": "C-band (SAR).",
        "modo": "SAR"
    },

    "AUTCUBE 2": {
        "descripcion": (
            "CubeSat de la Universidad de Augsburg (posiblemente). "
            "Proyecto educativo/demostración."
        ),
        "frecuencia": "UHF/VHF.",
        "modo": "Demostración / Telemetría"
    },

    "HJ-2E": {
        "descripcion": (
            "Huanjing (HJ) serie 2 (China). Satélite ambiental con cargas ópticas y SAR."
        ),
        "frecuencia": "X-band downlink (~8 GHz).",
        "modo": "Observación / Monitorización"
    },

    "GAOFEN-12 01": {
        "descripcion": (
            "Satélite de alta resolución chino (serie Gaofen). "
            "Probablemente SAR en C o X-band."
        ),
        "frecuencia": "Posible X-band para SAR, S-band TT&C.",
        "modo": "SAR"
    },

    "SHIYAN 4 (SY-4)": {
        "descripcion": (
            "Satélite chino de prueba (Shiyan = 'experimento'). "
            "Posible misión de validación tecnológica."
        ),
        "frecuencia": "No pública, S/X-band para telemetría.",
        "modo": "Pruebas tecnológicas"
    },

    "SJ-7": {
        "descripcion": (
            "Shijian-7, satélite chino científico/tecnológico. "
            "Datos concretos no divulgados."
        ),
        "frecuencia": "S-band (TT&C), X-band (posible).",
        "modo": "Ciencia/Ensayos"
    },

    "CFOSAT": {
        "descripcion": (
            "Chinese-French Oceanography SAT. Estudios del viento y olas marinas. "
            "Lanzado en 2018."
        ),
        "frecuencia": "X-band radar + S-band telemetría.",
        "modo": "Oceanografía"
    },

    "COSMOS 2496": {
        "descripcion": (
            "Satélite ruso (posiblemente un 'Rodnik' o misiones militares). "
            "Datos limitados."
        ),
        "frecuencia": "Clasificada, quizá en UHF/VHF/S-band.",
        "modo": "Militar/Comunicaciones"
    },

    "GONETS-M 8": {
        "descripcion": (
            "Satélite de mensajería/almacenamiento ruso (GONETS). Órbita LEO ~1400 km."
        ),
        "frecuencia": (
            "UHF ~250 MHz, L-band ~1.6 GHz o 2 GHz (varía)."
        ),
        "modo": "Almacenar y reenviar datos"
    },

    "CELESTIS-02 & TAURUS R/B": {
        "descripcion": (
            "Elemento conmemorativo (Celestis) y resto de cohete Taurus. "
            "No es un satélite operativo."
        ),
        "frecuencia": "No aplica (restos de cohete).",
        "modo": "No aplica"
    },

    "HINODE (SOLAR-B)": {
        "descripcion": (
            "Misión solar japonesa (JAXA) con colaboración NASA/ESA. "
            "Órbita heliosíncrona ~600 km."
        ),
        "frecuencia": "S-band (~2 GHz comandos), X-band datos (~8 GHz).",
        "modo": "Ciencia solar"
    },

    "LEMUR-2-ANDIS": {
        "descripcion": (
            "Uno de los numerosos LEMUR-2 (Spire). Recolección de datos AIS/ADS-B, "
            "teledetección. LEO ~500 km."
        ),
        "frecuencia": "UHF para downlink, S-band en algunos. ADS-B en 1090 MHz.",
        "modo": "AIS/ADS-B"
    },

    "GONETS-M 20": {
        "descripcion": (
            "Otro satélite GONETS (Rusia) para comunicaciones de órbita baja. "
            "~1400 km."
        ),
        "frecuencia": "UHF / L-band (varía).",
        "modo": "Almacenar y reenviar"
    },

    "PISAT": {
        "descripcion": (
            "Satélite universitario de PES University (India). "
            "Observación terrestre en órbita LEO."
        ),
        "frecuencia": "UHF/VHF (telemetría), S-band (posible).",
        "modo": "Telemetría / Imágenes"
    },

    "GONETS-M 10": {
        "descripcion": "Otro de la constelación GONETS de Rusia (~1400 km).",
        "frecuencia": "UHF, L-band (almacenar y reenviar).",
        "modo": "Mensajería satelital"
    },

    "COSMOS 2463": {
        "descripcion": (
            "Satélite ruso. Podría ser del sistema GLONASS-K o similar. "
            "Lanzado ~2009/2010."
        ),
        "frecuencia": "Posible L1/L2 (GLONASS) si es de navegación, o S-band militar.",
        "modo": "Navegación / Militar"
    },

    "2023-193A": {
        "descripcion": "Designación genérica (no real en 2023). Posible satélite desconocido.",
        "frecuencia": "Datos no disponibles.",
        "modo": "No aplica / Desconocido"
    },

    "YUNHAI-1 02": {
        "descripcion": (
            "Segundo satélite Yunhai-1 para meteorología/clima (China). "
            "Órbita ~700 km."
        ),
        "frecuencia": "X-band o S-band downlink.",
        "modo": "Meteorología"
    },

    "KINEIS-1B": {
        "descripcion": (
            "Otro satélite de la constelación Kineis IoT (Francia). "
            "LEO ~650 km."
        ),
        "frecuencia": "UHF/VHF para IoT, S-band TT&C.",
        "modo": "IoT (almacenar y reenviar)"
    },

    "SAPPHIRE": {
        "descripcion": (
            "Satélite canadiense de monitoreo espacial (SSA) ~786 km. "
            "Lanzado en 2013."
        ),
        "frecuencia": "S-band (~2 GHz) telemetría.",
        "modo": "Monitoreo de objetos (SSA)"
    },

    "SAOCOM 1A": {
        "descripcion": (
            "Satélite argentino de observación con SAR en banda L (~1.275 GHz). "
            "CONAE/INVAP."
        ),
        "frecuencia": "L-band SAR, S/X-band telemetría.",
        "modo": "SAR"
    },

    "APRIZESAT 2": {
        "descripcion": (
            "Pequeño satélite AprizeSat (EE.UU.) para comunicaciones de baja tasa, "
            "probablemente LEO ~700 km."
        ),
        "frecuencia": "UHF/VHF (almacén y reenvío).",
        "modo": "Mensajería / IoT"
    },

    "KINEIS-1C": {
        "descripcion": (
            "Otro satélite Kineis (Francia) para IoT. "
            "Parte de la constelación de 25 satélites planeados."
        ),
        "frecuencia": "UHF/VHF, S-band.",
        "modo": "IoT"
    },

    "ALSAT 1B": {
        "descripcion": (
            "Satélite argelino de observación (~670 km), lanzado en 2016. "
            "Colaboración con SSTL (UK)."
        ),
        "frecuencia": "X-band (downlink imágenes), S-band TT&C.",
        "modo": "Observación"
    },

    "NEXTSAT-2": {
        "descripcion": (
            "Possibly surcoreano (KARI) para demostraciones en LEO. "
            "Detalles limitados."
        ),
        "frecuencia": "S-band TT&C, X-band datos.",
        "modo": "Demostración tecnológica"
    },

    "TIANMU-1 14": {
        "descripcion": (
            "Satélite chino de medición meteorológica/ionosférica. "
            "Serie Tianmu para monitoreo del clima."
        ),
        "frecuencia": "S-band o X-band downlink.",
        "modo": "Meteorología / Ciencia"
    },

    "YUNHAI-1 04": {
        "descripcion": (
            "Otro satélite Yunhai-1 (China) para clima y atmósfera. "
            "Órbita ~700-800 km."
        ),
        "frecuencia": "X-band o S-band data.",
        "modo": "Meteorología"
    },

    "COSMOS 2523": {
        "descripcion": (
            "Satélite ruso (designación Cosmos). Podría ser militar o inspector. "
            "Datos no públicos."
        ),
        "frecuencia": "Clasificado (posible banda S).",
        "modo": "Militar / Inspección"
    },

    "QIANFAN-47": {
        "descripcion": (
            "Satélite chino 'Qianfan' (poco documentado). Probables pruebas LEO."
        ),
        "frecuencia": "No publicada. Posible S-band.",
        "modo": "Desconocido / Demostración"
    },

    "QIANFAN-41": {
        "descripcion": (
            "Otro satélite de la serie Qianfan. Datos muy limitados (China)."
        ),
        "frecuencia": "S-band (posible).",
        "modo": "Desconocido"
    },

    "IRIS": {
        "descripcion": (
            "Interface Region Imaging Spectrograph (NASA). Observa la cromosfera solar. "
            "Órbita ~600 km."
        ),
        "frecuencia": "S-band (comandos ~2 GHz), X-band (downlink datos).",
        "modo": "Astronomía solar"
    },

    "QIANFAN-46": {
        "descripcion": "Otro Qianfan chino, sin info oficial.",
        "frecuencia": "No disponible.",
        "modo": "Desconocido"
    },

    "OCEANSAT-2": {
        "descripcion": (
            "Satélite indio (ISRO) para oceanografía. Lanzado en 2009, "
            "lleva un dispersómetro y un radiómetro."
        ),
        "frecuencia": "X-band (downlink), S-band (TT&C).",
        "modo": "Oceanografía"
    },

    "QIANFAN-48": {
        "descripcion": "Otro satélite 'Qianfan'.",
        "frecuencia": "Desconocido.",
        "modo": "Desconocido"
    },

    "IRIDIUM 155": {
        "descripcion": (
            "Satélite de la constelación Iridium. "
            "Dedicado a telefonía satelital global."
        ),
        "frecuencia": "L-band usuario ~1.6 GHz, Ka-band inter-satélite.",
        "modo": "Telefonía/datos"
    },

    "SHIYAN 19 (SY-19)": {
        "descripcion": (
            "Satélite de experimentación tecnológica chino. "
            "Órbita LEO, detalles clasificados."
        ),
        "frecuencia": "S/X-band (telemetría).",
        "modo": "Pruebas tecnológicas"
    },

    "QIANFAN-44": {
        "descripcion": "De la serie Qianfan. Sin datos públicos concretos.",
        "frecuencia": "No disponible.",
        "modo": "Desconocido"
    },

    "YAOGAN-11": {
        "descripcion": (
            "Satélite chino de observación militar (Yaogan). "
            "Probablemente óptico o SAR en LEO ~600 km."
        ),
        "frecuencia": "Clasificado (X-band SAR o cámara óptica).",
        "modo": "Reconocimiento"
    },

    "COSMOS 2407": {
        "descripcion": (
            "Satélite ruso, podría ser un GLONASS o militar, lanzado ~2004."
        ),
        "frecuencia": "Si GLONASS, L1 (1602 MHz), L2 (1246 MHz). Si militar, clasificado.",
        "modo": "Navegación / Militar"
    },

    "IRIDIUM 156": {
        "descripcion": "Otro satélite Iridium (2ª o 3ª gen) en LEO.",
        "frecuencia": "L-band, Ka-band.",
        "modo": "Telefonía/datos"
    },

    "LEMUR-2-VALHALLA": {
        "descripcion": (
            "Miembro de la constelación Lemur-2 (Spire). AIS, ADS-B, "
            "radio occultation. LEO ~500-600 km."
        ),
        "frecuencia": "UHF, S-band, ADS-B (1090 MHz).",
        "modo": "AIS / ADS-B"
    },

    "ZHUHAI-1 OVS-02": {
        "descripcion": (
            "Satélite chino de la constelación Zhuhai-1. Carga óptica y/o hiperespectral."
        ),
        "frecuencia": "X-band (downlink imágenes).",
        "modo": "Observación"
    },

    "YAM-2": {
        "descripcion": (
            "Yet Another Mission 2 (York Space?). Podría ser un bus satelital en LEO."
        ),
        "frecuencia": "Datos limitados, S-band o X-band.",
        "modo": "Servicios satelitales"
    },

    "2025-013C": {
        "descripcion": "Designación hipotética de lanzamiento futuro. Sin datos.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "FENGYUN 3E": {
        "descripcion": (
            "Satélite meteorológico polar chino, parte de FY-3. "
            "Lanzado en 2021 (~836 km)."
        ),
        "frecuencia": "L/X-band (datos meteorológicos).",
        "modo": "Meteorología"
    },

    "SWARM C": {
        "descripcion": (
            "Parte de la misión SWARM (ESA) para estudiar el campo magnético terrestre. "
            "Órbita ~460-530 km."
        ),
        "frecuencia": "S-band (~2 GHz) TT&C, / X-band telemetría.",
        "modo": "Científico"
    },

    "SWARM A": {
        "descripcion": "Idéntico a SWARM C, conjunto de 3 satélites (A, B, C).",
        "frecuencia": "S-band TT&C, X-band datos.",
        "modo": "Científico"
    },

    "HUANJING 1A": {
        "descripcion": (
            "Satélite chino de monitorización ambiental. "
            "Órbita ~650 km. Lanzado ~2008."
        ),
        "frecuencia": "S/X-band (datos).",
        "modo": "Telemetría / Observación"
    },

    "HUANJING 1B": {
        "descripcion": "Compañero de 1A (HJ-1B), con sensor óptico/IR. LEO ~650 km.",
        "frecuencia": "X-band downlink.",
        "modo": "Observación / Monitorización"
    },

    "LUSAT (LO-19)": {
        "descripcion": (
            "Satélite argentino de radioaficionados (AMSAT-LU), lanzado en 1990. "
            "Aún en órbita, inactivo o señal muy débil."
        ),
        "frecuencia": "435 MHz (downlink), 145 MHz (uplink).",
        "modo": "Digital/packet (inactivo)."
    },

    "HAIYANG-2A": {
        "descripcion": (
            "Satélite chino de monitoreo oceánico. Radar altímetro, "
            "viento sobre océanos."
        ),
        "frecuencia": "X-band (datos), S-band TT&C.",
        "modo": "Oceanografía"
    },

    "SJ-12": {
        "descripcion": (
            "Shijian-12, satélite chino de experimentación. "
            "Puede haber realizado maniobras de rendezvous."
        ),
        "frecuencia": "S-band / X-band (clasificado).",
        "modo": "Pruebas tecnológicas"
    },

    "COSMOS 2497": {
        "descripcion": (
            "Satélite ruso. Podría ser inspector o un satélite 'Kosmos' de tareas militares."
        ),
        "frecuencia": "Clasificada (S-band).",
        "modo": "Militar"
    },

    "GAOFEN-3 02": {
        "descripcion": (
            "Satélite de observación radar (China). Banda C o X (varía). "
            "Resolución alta."
        ),
        "frecuencia": "C-band SAR (~5.4 GHz) probable.",
        "modo": "SAR"
    },

    "2024-174D": {
        "descripcion": "Designación hipotética. Datos no disponibles.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "NORSAT 1": {
        "descripcion": (
            "Satélite noruego para monitoreo de AIS en barcos. "
            "LEO ~600-650 km, lanzado 2017."
        ),
        "frecuencia": "AIS (162 MHz), VHF/UHF para telemetría.",
        "modo": "AIS / Observación"
    },

    "L-SAR 01A": {
        "descripcion": (
            "Satélite chino con SAR en banda L (~1.2 GHz). "
            "Parte de la serie Gaofen con radar."
        ),
        "frecuencia": "L-band SAR.",
        "modo": "SAR"
    },

    "HJ-2F": {
        "descripcion": "Otro satélite Huanjing-2 (clima y desastre), LEO ~650 km.",
        "frecuencia": "X-band (downlink).",
        "modo": "Observación / Medio ambiente"
    },

    "WNISAT-1": {
        "descripcion": (
            "Predecesor de WNISAT 1R. Proyecto de la Universidad de Wakayama. "
            "CubeSat ~600-700 km."
        ),
        "frecuencia": "UHF/VHF.",
        "modo": "Telemetría"
    },

    "CUBESAT XI-IV (CO-57)": {
        "descripcion": (
            "CubeSat japonés (Tokio U.) lanzado en 2003. Opera en VHF/UHF, APRS."
        ),
        "frecuencia": "436.847 MHz (downlink), 145 MHz (uplink).",
        "modo": "Radioaficionado"
    },

    "ASNARO-2": {
        "descripcion": (
            "Advanced Satellite with New system ARchitecture for Observation-2 (Japón). "
            "SAR en banda X."
        ),
        "frecuencia": "X-band (SAR ~9.65 GHz).",
        "modo": "SAR"
    },

    "EROS C3": {
        "descripcion": (
            "Satélite israelí de observación de la empresa ISI (ImageSat). "
            "Resolución submétrica."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Observación"
    },

    "COSMOS 2416": {
        "descripcion": (
            "Satélite ruso. Puede ser GLONASS o militar (lanzado ~2005)."
        ),
        "frecuencia": "Si GLONASS: L1, L2. O S-band si militar.",
        "modo": "Navegación / Militar"
    },

    "SUPERVIEW NEO-2 02": {
        "descripcion": (
            "Satélite chino de observación comercial de alta resolución (SuperView)."
        ),
        "frecuencia": "X-band (downlink imágenes).",
        "modo": "Observación óptica"
    },

    "PATHFINDER 1": {
        "descripcion": (
            "Posible satélite de pruebas LEO. Varias misiones se llaman 'Pathfinder'."
        ),
        "frecuencia": "Desconocido.",
        "modo": "Demostración"
    },

    "SHIYAN 6 01 (SY-6 01)": {
        "descripcion": "Satélite chino experimental Shiyan 6, primer ejemplar.",
        "frecuencia": "Clasificado.",
        "modo": "Ensayos"
    },

    "CALSPHERE 2": {
        "descripcion": (
            "Nombre de satélite calibración esférico (posiblemente de la USAF). "
            "Años 60-70. Podría ser un resto histórico."
        ),
        "frecuencia": "N/A",
        "modo": "Baliza calibración"
    },

    "NANOACE": {
        "descripcion": (
            "CubeSat de prueba (AeroCube?), lanzado ~2017. Demostración de sistemas."
        ),
        "frecuencia": "UHF/VHF.",
        "modo": "Demostración"
    },

    "PROBA-2": {
        "descripcion": (
            "Satélite de la ESA para experimentos solares y plasma. "
            "Órbita ~700 km."
        ),
        "frecuencia": "S-band (~2 GHz).",
        "modo": "Científico"
    },

    "JILIN-1": {
        "descripcion": (
            "Constelación china de observación comercial (Chang Guang). "
            "Varios satélites, alta resolución."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Observación"
    },

    "TIANMU-1 16": {
        "descripcion": "Satélite chino meteorológico/detección de desastres. LEO.",
        "frecuencia": "S/X-band.",
        "modo": "Meteorología"
    },

    "STARS-AO": {
        "descripcion": (
            "Posible CubeSat japonés (Kagawa Univ.?). 'STARS' series con tether experiment."
        ),
        "frecuencia": "UHF (aprox. 435 MHz).",
        "modo": "Experimento"
    },

    "LEMUR-2-GREENBERG": {
        "descripcion": "Otro satélite LEMUR-2 (Spire) para AIS, ADS-B, RO.",
        "frecuencia": "UHF, S-band, ADS-B (1090 MHz).",
        "modo": "AIS/ADS-B"
    },

    "NEOSSAT": {
        "descripcion": (
            "Canadian microsatellite (CSA) para búsqueda de asteroides cercanos. "
            "Órbita ~800 km."
        ),
        "frecuencia": "S-band (~2 GHz) TT&C.",
        "modo": "Astronomía"
    },

    "IRIDIUM 108": {
        "descripcion": (
            "Constelación Iridium. Telefonía satelital. LEO ~780 km."
        ),
        "frecuencia": "L-band, Ka-band.",
        "modo": "Datos / Telefonía"
    },

    "SMOS": {
        "descripcion": (
            "Soil Moisture and Ocean Salinity (ESA). Órbita heliosíncrona ~758 km."
        ),
        "frecuencia": "L-band (~1.4 GHz radiometría).",
        "modo": "Ciencia (microondas pasivas)"
    },

    "QIANFAN-38": {
        "descripcion": "Satélite chino de la serie Qianfan. Datos limitados.",
        "frecuencia": "No disponible",
        "modo": "Desconocido"
    },

    "CHEOPS": {
        "descripcion": (
            "CHaracterising ExOPlanet Satellite (ESA). Órbita ~700 km heliosíncrona. "
            "Observa tránsitos exoplanetarios."
        ),
        "frecuencia": "S-band (comandos), X-band (datos).",
        "modo": "Astronomía"
    },

    "2023-072D": {
        "descripcion": "Designación de lanzamiento hipotético en 2023.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "SNIPE 2": {
        "descripcion": (
            "Pequeño satélite (KASI, Corea) para estudios ionosféricos. "
            "Parte de una constelación SNIPE."
        ),
        "frecuencia": "S-band TT&C, UHF/VHF posible.",
        "modo": "Ciencia"
    },

    "INSPIRESAT-1": {
        "descripcion": (
            "CubeSat internacional (laboratorio SpaceKu). Órbita LEO. "
            "Estudios ionosféricos y SMC."
        ),
        "frecuencia": "UHF/VHF, posible S-band data.",
        "modo": "Ciencia"
    },

    "COSMOS 2408": {
        "descripcion": (
            "Satélite ruso. Puede ser GLONASS o militar. Lanzado ~2004. "
            "Designación Kosmos."
        ),
        "frecuencia": "Si GLONASS: L1/L2. Si militar: clasificado.",
        "modo": "Navegación / Militar"
    },

    "CARBONITE 1": {
        "descripcion": (
            "Pequeño satélite de observación británico (SSTL) con vídeo HD. "
            "Lanzado ~2015-2016."
        ),
        "frecuencia": "X-band (downlink vídeo).",
        "modo": "Observación"
    },

    "YAOGAN-15": {
        "descripcion": (
            "Satélite chino de reconocimiento. Puede ser radar o óptico. "
            "Lanzado ~2012."
        ),
        "frecuencia": "Clasificado (X-band).",
        "modo": "Reconocimiento"
    },

    "JILIN-1 07": {
        "descripcion": (
            "Parte de la constelación Jilin-1 (Chang Guang). Órbita ~535 km. "
            "Alta resolución."
        ),
        "frecuencia": "X-band (downlink imágenes).",
        "modo": "Observación"
    },

    "YAOGAN-32 B": {
        "descripcion": (
            "Par de satélites chinos Yaogan-32 (A y B) con posible ELINT / reconocimiento. "
            "Órbita LEO ~600 km."
        ),
        "frecuencia": "Posible banda L/S para ELINT.",
        "modo": "Reconocimiento"
    },

    "YAOGAN-32 A": {
        "descripcion": (
            "Compañero de Yaogan-32 B. Misión similar (ELINT)."
        ),
        "frecuencia": "Clasificado.",
        "modo": "Reconocimiento"
    },

    "LEMUR-2-CAEL": {
        "descripcion": "Otro cubesat LEMUR-2 de Spire (AIS/ADS-B).",
        "frecuencia": "UHF, S-band, ADS-B.",
        "modo": "AIS/ADS-B"
    },

    "2025-013E": {
        "descripcion": "Designación futura hipotética.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "YUNHAI-1 03": {
        "descripcion": (
            "Tercer satélite Yunhai-1, estudios meteorológicos, ionosféricos. "
            "China ~700 km."
        ),
        "frecuencia": "X-band or S-band.",
        "modo": "Meteorología"
    },

    "GAOFEN-8": {
        "descripcion": (
            "Satélite chino de alta resolución (Gaofen). Órbita LEO ~600-700 km."
        ),
        "frecuencia": "X-band (imágenes).",
        "modo": "Observación"
    },

    "OSCAR 7 (AO-7)": {
        "descripcion": (
            "Satélite de radioaficionados lanzado en 1974. Fue reactivado "
            "parcialmente (~2002)."
        ),
        "frecuencia": "145 MHz uplink, 435 MHz downlink (Mode B).",
        "modo": "SSB/CW"
    },

    "MOST": {
        "descripcion": (
            "Microvariability and Oscillations of STars (Canadá), telescopio espacial. "
            "Órbita ~820 km."
        ),
        "frecuencia": "S-band TT&C.",
        "modo": "Astronomía"
    },

    "CUTE-1 (CO-55)": {
        "descripcion": (
            "CubeSat japonés (Tokio Inst. Tech), lanzado 2003. "
            "Comunicaciones en VHF/UHF."
        ),
        "frecuencia": "VHF (145.990 MHz), UHF (435.xxx MHz).",
        "modo": "Radioaficionado"
    },

    "COSMOS 2517": {
        "descripcion": (
            "Satélite ruso designado Kosmos-2517, lanzado ~2015. "
            "Posiblemente militar."
        ),
        "frecuencia": "Clasificado.",
        "modo": "Militar"
    },

    "SUPERVIEW NEO-2 04": {
        "descripcion": (
            "Satélite chino de observación de la constelación SuperView. Alta resolución."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Observación"
    },

    "SUPERVIEW NEO-2 03": {
        "descripcion": "Similar al anterior, alta resolución, parte de SuperView NEO.",
        "frecuencia": "X-band downlink.",
        "modo": "Observación"
    },

    "SJ-11-05": {
        "descripcion": (
            "Shijian-11, satélite de prueba chinos, con varios ejemplares (5). "
            "Posible órbita ~700 km."
        ),
        "frecuencia": "S/X-band clasificado.",
        "modo": "Pruebas / Reconocimiento"
    },

    "MOMENTUS-X1": {
        "descripcion": (
            "Demostrador de servicios en órbita (Momentus). "
            "Posible bus de propulsión con agua/plasma."
        ),
        "frecuencia": "UHF/S-band telemetría.",
        "modo": "Servicio en órbita / Demostración"
    },

    "2024-174E": {
        "descripcion": "Designación hipotética futura.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "2024-174A": {
        "descripcion": "Designación hipotética futura.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "HAIYANG 4-01": {
        "descripcion": (
            "Satélite chino HY-4 en GEO para monitoreo oceánico, "
            "probablemente ~105°E."
        ),
        "frecuencia": "C/K_u?? (poca info).",
        "modo": "Oceanografía"
    },

    "XINGYUN-2 02": {
        "descripcion": (
            "Parte de la constelación XINGYUN IoT en LEO (~1000 km). "
            "Operado por CASIC."
        ),
        "frecuencia": "UHF/VHF, S-band IoT.",
        "modo": "IoT"
    },

    "HELIOS 2A": {
        "descripcion": (
            "Satélite militar óptico francés. Órbita ~700 km. Alta resolución."
        ),
        "frecuencia": "X-band cifrada (downlink).",
        "modo": "Reconocimiento"
    },

    "QIANFAN-52": {
        "descripcion": "Otro Qianfan chino. Sin datos.",
        "frecuencia": "N/D",
        "modo": "N/D"
    },

    "LEMUR-2-FURIAUS": {
        "descripcion": "Constelación Spire, AIS/ADS-B, LEMUR-2.",
        "frecuencia": "UHF, S-band, 1090 MHz.",
        "modo": "AIS/ADS-B"
    },

    "LEMUR-2-DEMBITZ": {
        "descripcion": "Similar a otros LEMUR-2. Órbita LEO ~500-600 km.",
        "frecuencia": "UHF, S-band.",
        "modo": "AIS/ADS-B"
    },

    "CSG-2": {
        "descripcion": (
            "COSMO-SkyMed segunda generación, satélite radar italiano. "
            "Banda X (~9.65 GHz)."
        ),
        "frecuencia": "X-band (SAR).",
        "modo": "SAR"
    },

    "COSMOS 2409": {
        "descripcion": (
            "Satélite ruso, ~2004. Podría ser GLONASS o militar."
        ),
        "frecuencia": "Si GLONASS: L1/L2; o clasificado.",
        "modo": "Navegación / Militar"
    },

    "OPTICUBE 04": {
        "descripcion": (
            "Posible CubeSat para demostraciones ópticas. Poca info pública."
        ),
        "frecuencia": "UHF/VHF.",
        "modo": "Demostración"
    },

    "LEMUR-2-MONSON": {
        "descripcion": "Otro LEMUR-2 de Spire, AIS/ADS-B.",
        "frecuencia": "UHF, S-band, 1090 ADS-B.",
        "modo": "AIS/ADS-B"
    },

    "ACS3": {
        "descripcion": (
            "Satélite de NASA o JAXA con vela solar (?), 'AeroCube Solar Sail 3' tal vez. "
            "Pruebas en LEO."
        ),
        "frecuencia": "UHF/S-band.",
        "modo": "Demostración"
    },

    "LEMUR-2-ZACHARY": {
        "descripcion": "Constelación LEMUR-2 (Spire). Mismos fines AIS/ADS-B.",
        "frecuencia": "UHF, S-band.",
        "modo": "AIS/ADS-B"
    },

    "YAOGAN-29": {
        "descripcion": (
            "Satélite chino de reconocimiento (posiblemente SAR u óptico). LEO ~620 km."
        ),
        "frecuencia": "Clasificado (X-band).",
        "modo": "Reconocimiento"
    },

    "ARIRANG-5 (KOMPSAT-5)": {
        "descripcion": (
            "Satélite surcoreano (KARI) con radar X-band (~1 m resolución). "
            "Órbita ~550 km."
        ),
        "frecuencia": "X-band (SAR), S-band TT&C.",
        "modo": "SAR"
    },

    "2023-072F": {
        "descripcion": "Designación hipotética de lanzamiento en 2023.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "ODIN": {
        "descripcion": (
            "Satélite sueco (2001) para estudios de atmósfera y astronomía submilimétrica. "
            "Órbita ~600 km."
        ),
        "frecuencia": "S-band TT&C, X-band datos.",
        "modo": "Ciencia atmósfera"
    },

    "SHIJIAN-6 01A (SJ-6 01A)": {
        "descripcion": (
            "Parte del dúo Shijian-6 01A/B. Satélites chinos de experimentación LEO."
        ),
        "frecuencia": "Clasificado, S/X-band.",
        "modo": "Tecnología / Reconocimiento"
    },

    "GRACE-FO 1": {
        "descripcion": (
            "Gravity Recovery and Climate Experiment Follow-On (NASA/GFZ). "
            "Dúo de satélites para medición de campos gravitatorios."
        ),
        "frecuencia": "K-band rango (24 GHz), S-band TT&C.",
        "modo": "Geodesia"
    },

    "GRACE-FO 2": {
        "descripcion": (
            "Segundo satélite GRACE-FO. Vuelan en formación ~490 km. "
        ),
        "frecuencia": "K-band (24 GHz) link, S-band TT&C.",
        "modo": "Geodesia"
    },

    "TIANMU-1 12": {
        "descripcion": (
            "Otro satélite Tianmu (China) para datos meteorológicos en LEO."
        ),
        "frecuencia": "S/X-band.",
        "modo": "Meteorología"
    },

    "NOAA 15": {
        "descripcion": (
            "Satélite meteorológico polar de EE.UU. (lanzado en 1998). "
            "APT en 137.620 MHz (si activo)."
        ),
        "frecuencia": "137.620 MHz APT, L-band HRPT (~1.7 GHz).",
        "modo": "APT/HRPT"
    },

    "2023-072G": {
        "descripcion": "Designación hipotética 2023.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "2023-072E": {
        "descripcion": "Designación hipotética 2023.",
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "COSMOS 2391": {
        "descripcion": (
            "Satélite ruso (posiblemente militar) lanzado ~2002. "
            "Pocos datos públicos."
        ),
        "frecuencia": "Clasificado.",
        "modo": "Militar"
    },

    "JIAOTONG VDES": {
        "descripcion": (
            "Posible satélite chino experimental para VDES (VHF Data Exchange System)."
        ),
        "frecuencia": "VHF (156-162 MHz?).",
        "modo": "VDES / AIS"
    },

    "GONETS-M 5": {
        "descripcion": (
            "Otro satélite Gonets, mensajería en órbita ~1400 km, ruso."
        ),
        "frecuencia": "UHF, L-band.",
        "modo": "Almacén y reenvío"
    },

    "M3MSAT": {
        "descripcion": (
            "Maritime Monitoring and Messaging Micro-Satellite (Canadá). "
            "Recibe AIS."
        ),
        "frecuencia": "AIS (162 MHz), S-band TT&C.",
        "modo": "AIS"
    },

    "KL-ALPHA B": {
        "descripcion": (
            "Satélite 'KL-Alpha'? Podría ser uno de Kepler Communications (KIPP, TARS)."
        ),
        "frecuencia": "Ku-band o S-band IoT.",
        "modo": "Datos"
    },

    "GONETS-M 17": {
        "descripcion": "Constelación Gonets (Rusia). Órbita LEO ~1400 km.",
        "frecuencia": "UHF / L-band.",
        "modo": "Mensajería"
    },

    "ARIRANG-2 (KOMPSAT-2)": {
        "descripcion": (
            "Satélite surcoreano de observación óptica, lanzado en 2006, "
            "resolución ~1 m."
        ),
        "frecuencia": "X-band (downlink), S-band TT&C.",
        "modo": "Observación"
    },

    "PAYLOAD B": {
        "descripcion": (
            "Designación genérica. No info concreta (podría ser un 'dummy payload')."
        ),
        "frecuencia": "N/A",
        "modo": "N/A"
    },

    "KL-ALPHA A": {
        "descripcion": (
            "Satélite 'Kepler' alpha (IoT). Conectividad en LEO ~575 km."
        ),
        "frecuencia": "S-band/Ku-band IoT.",
        "modo": "Datos"
    },

    "SHIJIAN-6 02A (SJ-6 02A)": {
        "descripcion": (
            "Satélite chino de experimentación (junto a 02B). "
            "Órbita LEO ~600 km."
        ),
        "frecuencia": "Posible S-band / X-band.",
        "modo": "Pruebas tecnológicas"
    },

    "SKYSAT-C12": {
        "descripcion": (
            "Satélite SkySat (Planet) con cámara submétrica. LEO ~500 km."
        ),
        "frecuencia": "X-band (downlink).",
        "modo": "Observación"
    },

    "SITRO-AIS 26": {
        "descripcion": (
            "Satélite AIS (posible, no muy documentado). "
            "Podría ser un cubesat ~500-600 km."
        ),
        "frecuencia": "AIS (161-162 MHz).",
        "modo": "AIS"
    }
}

def download_tle(url, output_file):

    print(f"Descargando TLE desde: {url}")
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        with open(output_file, 'wb') as f:
            f.write(resp.content)
        print(f"TLE descargado y guardado en '{output_file}'.\n")
    except Exception as e:
        print(f"Ocurrió un error al descargar el TLE: {e}")

def get_inclination_deg(satellite):

    return satellite.model.inclo * (180.0 / math.pi)

def find_next_n_mondays(n=3, tz=LOCAL_TZ):

    now_local = datetime.datetime.now(tz)
    current_wd = now_local.weekday()  # Monday=0, Tuesday=1, etc.
    days_until_monday = (0 - current_wd) % 7
    first_monday = now_local + datetime.timedelta(days=days_until_monday)
    # Ajustar a 00:00
    first_monday = first_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    return [first_monday + datetime.timedelta(weeks=i) for i in range(n)]

def main():

    download_tle(TLE_URL, TLE_LOCAL_FILE)


    print("Cargando TLE desde fichero local...\n")
    satellites = load.tle_file(TLE_LOCAL_FILE)
    print(f"Se han cargado {len(satellites)} satélites.\n")


    observer_location = wgs84.latlon(OBS_LAT, OBS_LON)
    next_mondays_local = find_next_n_mondays(14)


    monday_windows_utc = []
    ts = load.timescale()
    for monday_local in next_mondays_local:
        start_local = monday_local.replace(hour=18, minute=0, second=0, microsecond=0)
        end_local   = monday_local.replace(hour=21, minute=0, second=0, microsecond=0)

        start_utc = start_local.astimezone(pytz.utc)
        end_utc   = end_local.astimezone(pytz.utc)

        t_start = ts.from_datetime(start_utc)
        t_end   = ts.from_datetime(end_utc)
        monday_windows_utc.append((t_start, t_end))

    resultados = []


    for sat in satellites:
        sat_name = sat.name
        incl_deg = get_inclination_deg(sat)

        for (t_start, t_end) in monday_windows_utc:

            tiempos_eventos, tipos_eventos = sat.find_events(
                observer_location, t_start, t_end, altitude_degrees=0.0
            )

            pase_actual = {}
            max_elev_deg = 0.0
            aos_local_dt = None


            for (t_evento, tipo_evento) in zip(tiempos_eventos, tipos_eventos):
                dt_utc = t_evento.utc_datetime().replace(tzinfo=datetime.timezone.utc)
                dt_local = dt_utc.astimezone(LOCAL_TZ)

                if tipo_evento == 0:
                    pase_actual = {}
                    pase_actual['objeto'] = sat_name
                    aos_local_dt = dt_local
                    pase_actual['aos_local_dt'] = aos_local_dt
                    pase_actual['aos_local_str'] = dt_local.strftime('%Y-%m-%d %H:%M:%S')
                    max_elev_deg = 0.0

                elif tipo_evento == 1:

                    difference = sat - observer_location
                    topocentrico = difference.at(t_evento)
                    alt, az, distance = topocentrico.altaz()
                    max_elev_deg = alt.degrees


                    geocentrico = sat.at(t_evento)
                    dist_centro_tierra = geocentrico.distance().km
                    altitud_km = dist_centro_tierra - 6378.0
                    pase_actual['altitud_km'] = f"{altitud_km:.1f}"

                elif tipo_evento == 2:
                    if aos_local_dt is not None:
                        pase_actual['los_local_str'] = dt_local.strftime('%Y-%m-%d %H:%M:%S')
                        pase_actual['inclinacion_deg'] = f"{incl_deg:.2f}"


                        if (incl_deg >= MIN_INCL_DEG) and (MIN_ELEV_DEG <= max_elev_deg <= MAX_ELEV_DEG):

                            info_satelite = {
                                "descripcion": "No disponible",
                                "frecuencia": "Desconocida",
                                "modo": "Desconocido",
                            }
                            for known_name, satdata in SAT_INFO.items():
                                if known_name.upper() in sat_name.upper():
                                    info_satelite = satdata
                                    break

                            pase_actual['descripcion'] = info_satelite.get("descripcion", "No disponible")
                            pase_actual['frecuencia'] = info_satelite.get("frecuencia", "Desconocida")
                            pase_actual['modo'] = info_satelite.get("modo", "Desconocido")
                            pase_actual['elev_max_deg'] = f"{max_elev_deg:.1f}"


                            if (aos_local_dt.weekday() == 0) and (18 <= aos_local_dt.hour < 21):
                                resultados.append(pase_actual)


                    pase_actual = {}
                    aos_local_dt = None


    resultados.sort(key=lambda x: x['aos_local_dt'])


    if not resultados:
        print(f"No se encontraron pases los próximos 3 lunes entre 18:00 y 21:00 (hora local) "
              f"con inclinación >= {MIN_INCL_DEG}° y elevación {MIN_ELEV_DEG}°–{MAX_ELEV_DEG}°.")
        return

    print("Pases encontrados en los próximos 3 lunes (18:00–21:00 local), "
          f"inclinación >= {MIN_INCL_DEG}°, elev. máx. {MIN_ELEV_DEG}°–{MAX_ELEV_DEG}°:\n")


    encabezado = (
        "{:<35} | {:<30} | {:<8} | {:<8} | {:<8} | {:<15} | {:<10} | {}"
    )
    print(encabezado.format(
        "Objeto",
        "Rango Horario Local [AOS→LOS]",
        "Alt(km)",
        "Inc.(°)",
        "MaxEl",
        "Frecuencia",
        "Modo",
        "Descripción"
    ))
    print("-" * 150)

    for p in resultados:
        aos_str = p.get('aos_local_str', 'N/A')
        los_str = p.get('los_local_str', 'N/A')


        try:
            date_aos = aos_str.split(' ')[0]
            time_aos = aos_str.split(' ')[1] if ' ' in aos_str else ''
            date_los = los_str.split(' ')[0]
            time_los = los_str.split(' ')[1] if ' ' in los_str else ''

            if date_aos == date_los:
                rango_horario_local = f"{date_aos} {time_aos} → {time_los}"
            else:
                rango_horario_local = f"{aos_str} → {los_str}"
        except:
            rango_horario_local = f"{aos_str} → {los_str}"

        print(encabezado.format(
            p.get('objeto', 'N/A'),
            rango_horario_local,
            p.get('altitud_km', 'N/A'),
            p.get('inclinacion_deg', 'N/A'),
            p.get('elev_max_deg', 'N/A'),
            p.get('frecuencia', 'N/A'),
            p.get('modo', 'N/A'),
            p.get('descripcion', 'N/A'),
        ))

    print("Hasta aquí todo bien")
    # Define the output CSV file path
    csv_file_path = "./satellite_passes.csv"

    # Define the header for the CSV file
    csv_headers = [
        "Objeto", "Fecha/hora inicio (AOS)", "Fecha/hora fin (LOS)",
        "Frecuencia", "Modo", "Altura (km)", "Inclinación"
    ]

    # Writing results to a CSV file
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write header
        writer.writerow(csv_headers)

        for p in resultados:
            writer.writerow([
                p.get('objeto', 'N/A'),
                p.get('aos_local_str', 'N/A'),
                p.get('los_local_str', 'N/A'),
                p.get('frecuencia', 'N/A'),
                p.get('modo', 'N/A'),
                p.get('altitud_km', 'N/A'),
                p.get('inclinacion_deg', 'N/A'),
            ])
    # Provide the file path for downloading
    csv_file_path

    # Count occurrences of each satellite in `resultados`
    satellite_counts = Counter(p["objeto"] for p in resultados)

    # Find the most frequent satellite
    most_frequent_satellite_name, _ = satellite_counts.most_common(1)[0]

    # Define the new CSV file path for the most frequent satellite
    csv_most_frequent_sat_path = "./satellite_most_frequent.csv"

    # Extract only the relevant passes
    filtered_results = [p for p in resultados if p["objeto"] == most_frequent_satellite_name]

    # Writing the filtered results to a new CSV file
    with open(csv_most_frequent_sat_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write header
        writer.writerow(csv_headers)

        # Write each row for the most frequent satellite
        for p in filtered_results:
            writer.writerow([
                p.get('objeto', 'N/A'),
                p.get('aos_local_str', 'N/A'),
                p.get('los_local_str', 'N/A'),
                p.get('frecuencia', 'N/A'),
                p.get('modo', 'N/A'),
                p.get('altitud_km', 'N/A'),
                p.get('inclinacion_deg', 'N/A'),
            ])

    # Provide the file path for downloading
    csv_most_frequent_sat_path

    return resultados

if __name__ == "__main__":
    resultados = main()


Descargando TLE desde: https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle
TLE descargado y guardado en 'satellites.tle'.

Cargando TLE desde fichero local...

Se han cargado 11018 satélites.

Pases encontrados en los próximos 3 lunes (18:00–21:00 local), inclinación >= 70.0°, elev. máx. 70.0°–90.0°:

Objeto                              | Rango Horario Local [AOS→LOS]  | Alt(km)  | Inc.(°)  | MaxEl    | Frecuencia      | Modo       | Descripción
------------------------------------------------------------------------------------------------------------------------------------------------------
KINEIS-5E                           | 2025-02-17 18:00:23 → 18:13:43 | 645.0    | 97.95    | 73.3     | UHF/VHF (mensajería IoT), S-band (posible TT&C). | IoT/Telemetría | Satélite de la constelación Kineis (Francia) para IoT y conectividad global, en órbita LEO ~650 km.
ONEWEB-0362                         | 2025-02-17 18:01:47 → 18:21:34 | 1181.5   | 87.92    | 87.2     | Ku-band

In [9]:

    # Define the header for the CSV file
    csv_headers = [
        "Objeto", "Fecha/hora inicio (AOS)", "Fecha/hora fin (LOS)",
        "Frecuencia", "Modo", "Altura (km)", "Inclinación"
    ]

    # Count occurrences of each satellite in `resultados`
    satellite_counts = Counter(p["objeto"] for p in resultados)

    # Find the most frequent satellite
    most_frequent_satellite_name, _ = satellite_counts.most_common(1)[0]

    # Define the new CSV file path for the most frequent satellite
    csv_most_frequent_sat_path = "./satellite_most_frequent.csv"

    # Extract only the relevant passes
    filtered_results = [p for p in resultados if p["objeto"] == most_frequent_satellite_name]

    # Writing the filtered results to a new CSV file
    with open(csv_most_frequent_sat_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write header
        writer.writerow(csv_headers)

        # Write each row for the most frequent satellite
        for p in filtered_results:
            writer.writerow([
                p.get('objeto', 'N/A'),
                p.get('aos_local_str', 'N/A'),
                p.get('los_local_str', 'N/A'),
                p.get('frecuencia', 'N/A'),
                p.get('modo', 'N/A'),
                p.get('altitud_km', 'N/A'),
                p.get('inclinacion_deg', 'N/A'),
            ])

    # Provide the file path for downloading
    csv_most_frequent_sat_path


'./satellite_most_frequent.csv'

In [10]:
# Re-import necessary libraries after execution state reset
import pandas as pd

# Load the uploaded CSV file
uploaded_csv_path = "./satellite_passes.csv"

# Read the CSV file into a DataFrame
df_passes = pd.read_csv(uploaded_csv_path, encoding='utf-8')

# Determine the most frequent satellite
most_frequent_satellite_name = df_passes["Objeto"].mode()[0]

# Filter the DataFrame for only the most frequent satellite's passes
df_most_frequent = df_passes[df_passes["Objeto"] == most_frequent_satellite_name]

# Define the new CSV file path for the most frequent satellite
csv_most_frequent_sat_path = "./satellite_most_frequent_from_uploaded.csv"

# Save the filtered DataFrame to a new CSV file
df_most_frequent.to_csv(csv_most_frequent_sat_path, index=False, encoding='utf-8')

# Provide the file path for downloading
csv_most_frequent_sat_path

'./satellite_most_frequent_from_uploaded.csv'

In [12]:
# Convert AOS time to date format to count unique passes per day
df_passes["Fecha"] = pd.to_datetime(df_passes["Fecha/hora inicio (AOS)"]).dt.date

# Count unique passes per day per satellite
unique_passes_per_day = df_passes.groupby(["Objeto", "Fecha"]).size().reset_index(name="Count")

# Determine the satellite with the most unique passes per day
most_frequent_satellite_name = unique_passes_per_day["Objeto"].mode()[0]

# Filter the DataFrame for only the most frequent satellite's unique passes (one per day)
df_most_frequent = df_passes[df_passes["Objeto"] == most_frequent_satellite_name].drop_duplicates(subset=["Fecha"])

# Define the new CSV file path for the most frequent satellite with unique daily passes
csv_most_frequent_sat_unique_path = "./satellite_most_frequent_unique_daily.csv"

# Save the filtered DataFrame to a new CSV file
df_most_frequent.to_csv(csv_most_frequent_sat_unique_path, index=False, encoding='utf-8')

# Provide the file path for downloading
csv_most_frequent_sat_unique_path

'./satellite_most_frequent_unique_daily.csv'